In [4]:
import pandas as pd

# defaults
neighbours = "adjacency.csv"

# reads the csv file and converts it into a dataframe
def read_neighborgs(file_name: str) -> pd.DataFrame:
    return pd.read_csv(file_name)

# gets a list of counties and returns the uniques
def get_unique_counties(counties: list) -> list:
    return list(set(counties))

# gets a list of neighbors and returns the uniques
def get_unique_neighbors(neighbours: list) -> list:
    return list(set(neighbours))

# finds the shortest path from the starting locations to the goal location
# using a search method - an integer
# details_output = {F: not showing the first iteration, T: showing the first iteration}
def find_path(starting_locations ,goal_locations ,search_method: int, detail_output: bool):
    pass

if __name__ == "__main__":
    raw_df = read_neighborgs(neighbours)
    counties = get_unique_counties(raw_df['countyname'])
    neighbours = get_unique_neighbors(raw_df['neighborname'])


3232
3240
